# AutoMM for Text + Tabular - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/master/docs/tutorials/multimodal/multimodal_prediction/multimodal_text_tabular.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/master/docs/tutorials/multimodal/multimodal_prediction/multimodal_text_tabular.ipynb)



In many applications, text data may be mixed with numeric/categorical data. 
AutoGluon's `MultiModalPredictor` can train a single neural network that jointly operates on multiple feature types, 
including text, categorical, and numerical columns. The general idea is to embed the text, categorical and numeric fields 
separately and fuse these features across modalities. This tutorial demonstrates such an application.

In [ ]:
# !pip install autogluon.multimodal
# colab에서 적용가능한 최신버전
# !pip install autogluon==0.8.1

In [1]:
import numpy as np
import pandas as pd
import warnings
import os

warnings.filterwarnings('ignore')
np.random.seed(123)

In [ ]:
# !python3 -m pip install openpyxl

## Book Price Prediction Data

For demonstration, we use the book price prediction dataset from the [MachineHack Book Price Prediction Hackathon](https://www.machinehack.com/hackathons/predict_the_price_of_books/overview). Our goal is to predict a book's price given various features like its author, the abstract, the book's rating, etc.

In [7]:
!mkdir -p price_of_books
# !wget https://automl-mm-bench.s3.amazonaws.com/machine_hack_competitions/predict_the_price_of_books/Data.zip -O price_of_books/Data.zip
!curl https://automl-mm-bench.s3.amazonaws.com/machine_hack_competitions/predict_the_price_of_books/Data.zip -o price_of_books/Data.zip
!cd price_of_books && unzip -o Data.zip
!ls price_of_books/Participants_Data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3439k  100 3439k    0     0   653k      0  0:00:05  0:00:05 --:--:--  781k
Archive:  Data.zip
  inflating: Participants_Data/Data_Test.xlsx  
  inflating: Participants_Data/Data_Train.xlsx  
  inflating: Participants_Data/Sample_Submission.xlsx  
Data_Test.xlsx         Data_Train.xlsx        Sample_Submission.xlsx


In [8]:
train_df = pd.read_excel(os.path.join('price_of_books', 'Participants_Data', 'Data_Train.xlsx'), engine='openpyxl')
train_df.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


We do some basic preprocessing to convert `Reviews` and `Ratings` in the data table to numeric values, and we transform prices to a log-scale.

In [9]:
def preprocess(df):
    df = df.copy(deep=True)
    df.loc[:, 'Reviews'] = pd.to_numeric(df['Reviews'].apply(lambda ele: ele[:-len(' out of 5 stars')]))
    df.loc[:, 'Ratings'] = pd.to_numeric(df['Ratings'].apply(lambda ele: ele.replace(',', '')[:-len(' customer reviews')]))
    df.loc[:, 'Price'] = np.log(df['Price'] + 1)
    return df

In [10]:
train_subsample_size = 1500  # subsample for faster demo, you can try setting to larger values
test_subsample_size = 5
train_df = preprocess(train_df)
train_data = train_df.iloc[100:].sample(train_subsample_size, random_state=123)
test_data = train_df.iloc[:100].sample(test_subsample_size, random_state=245)
train_data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
949,Furious Hours,Casey Cep,"Paperback,– 1 Jun 2019",4.0,NaN,‘It’s been a long time since I picked up a boo...,True Accounts (Books),"Biographies, Diaries & True Accounts",5.743003
5504,REST API Design Rulebook,Mark Masse,"Paperback,– 7 Nov 2011",5.0,NaN,"In todays market, where rival web services com...","Computing, Internet & Digital Media (Books)","Computing, Internet & Digital Media",5.786897
5856,The Atlantropa Articles: A Novel,Cody Franklin,"Paperback,– Import, 1 Nov 2018",4.5,2.0,#1 Amazon Best Seller! Dystopian Alternate His...,Action & Adventure (Books),Romance,6.893656
4137,Hickory Dickory Dock (Poirot),Agatha Christie,"Paperback,– 5 Oct 2017",4.3,21.0,There’s more than petty theft going on in a Lo...,Action & Adventure (Books),"Crime, Thriller & Mystery",5.192957
3205,The Stanley Kubrick Archives (Bibliotheca Univ...,Alison Castle,"Hardcover,– 21 Aug 2016",4.6,3.0,"In 1968, when Stanley Kubrick was asked to com...",Cinema & Broadcast (Books),Humour,6.889591


## Training

We can simply create a MultiModalPredictor and call `predictor.fit()` to train a model that operates on across all types of features. 
Internally, the neural network will be automatically generated based on the inferred data type of each feature column. 
To save time, we subsample the data and only train for three minutes.

In [11]:
from autogluon.multimodal import MultiModalPredictor
import uuid

time_limit = 3 * 60  # set to larger value in your applications
model_path = f"./tmp/{uuid.uuid4().hex}-automm_text_book_price_prediction"
predictor = MultiModalPredictor(label='Price', path=model_path)
predictor.fit(train_data, time_limit=time_limit)

=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 23.2.0: Wed Nov 15 21:54:10 PST 2023; root:xnu-10002.61.3~2/RELEASE_X86_64
CPU Count:          16
Pytorch Version:    2.0.0.post104
CUDA Version:       CUDA is not available
Memory Avail:       38.80 GB / 64.00 GB (60.6%)
Disk Space Avail:   469.13 GB / 931.55 GB (50.4%)
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (9.115699967822062, 3.6109179126442243, 6.02567, 0.7694)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a 

Epoch 0:   8%|▊         | 12/150 [02:58<34:07, 14.84s/it]                  

Time limit reached. Elapsed time is 0:03:11. Signaling Trainer to stop.


Epoch 0:   9%|▊         | 13/150 [03:11<33:38, 14.74s/it]
Validation: 0it [00:00, ?it/s]
Epoch 0:   9%|▊         | 13/150 [06:01<1:03:33, 27.84s/it]             


AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/Users/elnath/004_deep_learning/AutoGloun-Official/v1_0_0/docs/tutorials/multimodal/multimodal_prediction/tmp/c232e5242ef54d3f9b8f014fe4b81fc3-automm_text_book_price_prediction")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




## Prediction

We can easily obtain predictions and extract data embeddings using the MultiModalPredictor.

In [12]:
predictions = predictor.predict(test_data)
print('Predictions:')
print('------------')
print(np.exp(predictions) - 1)
print()
print('True Value:')
print('------------')
print(np.exp(test_data['Price']) - 1)


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.15it/s]
Predictions:
------------
1     391.533691
31    553.008545
19    443.000916
45    223.713638
82    156.661316
Name: Price, dtype: float32

True Value:
------------
1     202.93
31    799.00
19    352.00
45    395.10
82    409.00
Name: Price, dtype: float64


In [13]:
performance = predictor.evaluate(test_data)
print(performance)

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]
{'rmse': 0.6084602397897769}


In [14]:
embeddings = predictor.extract_embedding(test_data)
embeddings.shape

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s]


(5, 128)

In [15]:
embeddings

array([[ 1.97017276e+00,  4.42579478e-01, -1.22913346e-02,
        -1.32129593e-02,  6.30735159e-02,  2.38969803e-01,
        -1.36235947e-04,  7.87246108e-01,  1.23061907e+00,
        -1.74127314e-02, -3.10689735e-04,  9.34759796e-01,
        -2.25785468e-02,  8.15839708e-01,  5.31397998e-01,
        -2.09610164e-03, -1.36573799e-04, -1.45463608e-02,
         7.02485085e-01,  2.59249115e+00,  7.82938451e-02,
        -1.85439289e-02,  1.13738513e+00,  5.96459031e-01,
        -3.61794163e-03, -1.17434394e-02, -1.25883669e-02,
        -1.44523932e-02,  1.15524507e+00,  1.47011817e+00,
         1.26489735e+00,  2.72900283e-01,  6.36970580e-01,
         1.48887372e+00, -1.73355415e-02,  6.12201393e-01,
         8.57340991e-01,  4.31806087e-01,  2.04117465e+00,
        -2.11990438e-02, -1.57048143e-02,  9.30061996e-01,
         9.02345717e-01,  9.50942159e-01,  4.36750412e-01,
        -6.08283794e-03,  7.89462328e-01,  2.48820925e+00,
         2.05588770e+00, -2.68533966e-03, -2.07936615e-0

In [17]:
predictor.fit_summary(verbosity=4, show_plot=True)

{'val_rmse': 1.3714981079101562, 'training_time': 403.9984030723572}


## Other Examples

You may go to [AutoMM Examples](https://github.com/autogluon/autogluon/tree/master/examples/automm) to explore other examples about AutoMM.

## Customization
To learn how to customize AutoMM, please refer to [Customize AutoMM](../advanced_topics/customization.ipynb).